In [ ]:
import csv
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import scipy
import seaborn as sns
sns.set()
sns.set_style("white")
plt.rcParams.update({'font.size': 20})
from collections import OrderedDict
import math
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern, RationalQuadratic, ExpSineSquared

In [ ]:
plt.rc('xtick', labelsize=20)    
plt.rc('ytick', labelsize=20)  

In [ ]:
def getData(df):
    # create array of pairs of all of these
    times = np.zeros(len(df))
    data = []
    for i, row in df.iterrows():
        data.append([])
#         for count, joint_idx in enumerate(range(0,3*3,3)):
#             data[i].append(list(row[2+joint_idx:2+joint_idx+3]))
        for count, joint_idx in enumerate(range(0,50*4,4)):
            if row[11+joint_idx+3] <= 0:
                data[i].append([])
            else:
                data[i].append(list(row[11+joint_idx:11+joint_idx+3]))

        times[i] = row[1]
    return np.array(data), np.array(times)

In [ ]:
def get3Ddata(data, times,  marker_idx):
    X = []
    y = []
    
    relevant = np.array(data)[:,marker_idx]
    for i, marker in enumerate(relevant):
        if len(marker) == 0:
            continue
        y.append(marker)
        X.append(times[i])
    y = np.array(y)
    X = np.array(X).reshape(-1,1)
    return X,y

In [ ]:
train_data1, train_times1 = getData(pd.read_csv('data_GP/data_GP/AG/block1-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213203046-59968-right-speed_0.500.csv', delimiter=','))
train_data2, train_times2 = getData(pd.read_csv('data_GP/data_GP/AG/block2-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213204004-59968-right-speed_0.500.csv', delimiter=','))
train_data3, train_times3 = getData(pd.read_csv('data_GP/data_GP/AG/block3-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213204208-59968-right-speed_0.500.csv', delimiter=','))
train_data4, train_times4  = getData(pd.read_csv('data_GP/data_GP/AG/block4-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213204925-59968-right-speed_0.500.csv', delimiter=','))
test_data1, test_times1   = getData(pd.read_csv('data_GP/data_GP/AG/block5-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213210121-59968-right-speed_0.500.csv', delimiter=','))

In [ ]:
marker = 8
Xwhole1, ywhole1 = get3Ddata(train_data1, train_times1, marker)
Xwhole2, ywhole2 = get3Ddata(train_data2, train_times2, marker)
Xwhole4, ywhole4 = get3Ddata(train_data3, train_times3, marker)
Xwhole5, ywhole5 = get3Ddata(train_data4, train_times4, marker)
Xwhole3, ywhole3 = get3Ddata(test_data1, test_times1, marker)

%matplotlib
sns.set_palette("pastel")
pal = sns.color_palette("pastel")
colors = pal.as_hex()


fig=plt.figure(figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')
print(ywhole1.shape)
ax.plot(ywhole2[:,0],ywhole2[:,1],ywhole2[:,2],lw=5)
ax.plot(ywhole3[:,0],ywhole3[:,1],ywhole3[:,2],lw=5)
ax.plot(ywhole4[:,0],ywhole4[:,1],ywhole4[:,2],lw=5)
# ax.scatter(Xwhole5,ywhole5[:,0],ywhole5[:,1],c=ywhole5[:,2], cmap=plt.hot())

# plt.scatter(Xwhole2,ywhole2)
# plt.scatter(Xwhole3, ywhole3)
# plt.scatter(Xwhole4, ywhole4)
# plt.scatter(Xwhole5, ywhole5)

In [ ]:
def fitAndPlotGP(Xtrain,ytrain,Xtest,ytest, kernel,times, title='', graph=True):
    ytrain = ytrain.copy()
    y_no_noise = ytrain.copy()
#     dy = .05 * np.random.random(y.shape)
#     noise = np.random.normal(0, dy)
#     ytrain += noise
    
    
    
    gp = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=0, normalize_y=True)
    gp.fit(Xtrain,ytrain)
    pred, std = gp.predict(Xtest, return_std=True)
    
    testCoef = gp.score(Xtest,ytest)
#     print('Test Coefficient',gp.score(Xtest,ytest))
        
    sq_res = np.sum((ytest-pred)**2)
    pred, std = gp.predict(times, return_std=True)

        
    if graph:
#         plt.axes(projection='3d')
        plt.figure(figsize=(24, 18),  dpi=80,facecolor='w', edgecolor='k')
        plt.plot(Xtest,ytest[:,0], colors[2], label='test data', linewidth=5)
        plt.plot(Xtest,ytest[:,1], colors[2], linewidth=5)
        plt.plot(Xtest,ytest[:,2], colors[2], linewidth=5)

        plt.plot(Xtrain, y_no_noise[:,0], 'o',color=colors[0], ms=5, label='x traning')
        plt.plot(Xtrain, y_no_noise[:,1], 'v',color=colors[5], ms=5, label='y traning')
        plt.plot(Xtrain, y_no_noise[:,2], 'X',color=colors[6], ms=5, label='z traning')

#         plt.fill_between(times.flat, pred-1.96*std, pred+1.96*std, color=colors[3], alpha=.2)
        print(std.shape)
        plt.fill_between(times.flat, pred[:,0]-1.96*std, pred[:,0]+1.96*std, color=colors[1], alpha=.2)
        plt.fill_between(times.flat, pred[:,1]-1.96*std, pred[:,1]+1.96*std, color=colors[3], alpha=.2)
        plt.fill_between(times.flat, pred[:,2]-1.96*std, pred[:,2]+1.96*std, color=colors[4], alpha=.2)

        plt.plot(times, pred[:,0], colors[1], lw=2, label='x mean', linewidth=5)
        plt.plot(times, pred[:,1], colors[3], lw=2, label='y mean', linewidth=5)
        plt.plot(times, pred[:,2], colors[4], lw=2, label='z mean', linewidth=5)

        
        plt.axis([0, times.flatten().max(), min(ytest.min(),pred.min())-.2, max(ytest.max(),pred.max())+.2])
        plt.title(title+'Sum of Squared Residuals: ' + str(sq_res) + '\n' + str(gp.kernel_),fontsize=35)

        plt.xlabel('time',fontsize=30)
        plt.ylabel('coordinate location',fontsize=30)
        plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1,  scatterpoints=1, prop={'size': 20},  title_fontsize=20)
        plt.savefig('3d '+title)
        return sq_res
    return sq_res, pred, std, gp.kernel_

# Global Kernel Function

In [ ]:




marker = 8
# Xwhole1, ywhole1 = get1Ddata(train_data1, train_times1, marker, coord)
Xwhole2, ywhole2 = get3Ddata(train_data2, train_times2, marker)
Xwhole4, ywhole4 = get3Ddata(train_data3, train_times3, marker)
Xwhole5, ywhole5 = get3Ddata(train_data4, train_times4, marker)
Xwhole3, ywhole3 = get3Ddata(test_data1, test_times1, marker)

Xwhole = np.concatenate([Xwhole2,Xwhole3,Xwhole4])
ywhole = np.concatenate([ywhole2,ywhole3,ywhole4])



print(Xwhole.shape)
size = int(len(Xwhole)/8)
# size = 500
# between = np.random.randint(len(Xwhole), size=size)
between = np.random.choice(np.arange(len(Xwhole)), size=size, replace=False)

X = np.array([Xwhole[i] for i in between])
y = np.array([ywhole[i] for i in between])


print(X.shape)

# temporal_X = np.arange(i*skips, i*skips+skips,.05).reshape(-1,1)

# y = np.concatenate([y1,y2])

# kernel = 1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1)+ WhiteKernel() + DotProduct() 
kernel = ConstantKernel(constant_value=.25, constant_value_bounds=(0,10)) + RBF() + WhiteKernel(noise_level=3.0, noise_level_bounds=(0,10))

last_time = sorted(Xwhole.flat)[-1]
temporal_X = np.arange(0, last_time,.05).reshape(-1,1)
# kernel = RBF()


fitAndPlotGP(X,y,Xwhole5,ywhole5,kernel,temporal_X, title="marker {} using global kernel\n".format(marker), graph=True)
plt.show()

# Local Kernel Functions

In [ ]:
def getTemporalSlice(start, end, data, times):
    times = times.flatten()
    data = data[times.argsort()]

    times = times[times.argsort()]
    idxs = np.where((start < times) & (times <= end))
    return times[idxs].reshape(-1,1), data[idxs]

def avg_group3d(A, B):
    df = pd.DataFrame({'A':A, 'Bx':B[:,0], 'By':B[:,1], 'Bzx':B[:,2]})
    res = df.groupby('A').mean()
    keys = np.array(list(df.groupby('A').groups.keys()))
    ans = res.to_numpy()
    return keys,ans
def avg_group1d(A, B):
    df = pd.DataFrame({'A':A, 'B':B})
    res = df.groupby('A').mean()
    keys = np.array(list(df.groupby('A').groups.keys()))
    ans = res.to_numpy().flatten()
    return keys,ans
print(stds.shape)
t1,t2=avg_group1d(times,stds)
print(t1.shape)
print(t2.shape)
plt.scatter(t1,t2)
plt.show()
print(t1)
print(t2)


In [ ]:
%matplotlib
kernel = ConstantKernel(constant_value=.25, constant_value_bounds=(0,10)) + RBF() + WhiteKernel(noise_level=3.0, noise_level_bounds=(0,10))
last_time = sorted(Xwhole.flat)[-1]
print(last_time)

plotting = True
res_list = []
coef_list = []
# for kernel in kernels:
plt.ioff()
all_temporal_X = np.arange(0, last_time,.001).reshape(-1,1)

for repeat in range(1):
    for iteration in [1]:
#     for iteration in [2]:

        delta = .1
        window_size = 1
        
        start = np.arange(0, last_time-2*delta, delta)
        end = np.arange(window_size, last_time+delta, delta)

        
        
        print("repeat number {} with delta {} and window_size {}".format(repeat, delta, window_size))

        
        preds = np.empty(shape=(0,3))
        stds = np.array([])
        Xtests = np.array([])
        ytests = np.empty(shape=(0,3))
        times = np.array([])
        Xs = np.array([])
        ys= np.empty(shape=(0,3))
        sumSqRes = 0
        hyperparams= np.array(kernel.theta)[np.newaxis,:]
        for i in range(min(len(end), len(start))):
            X,y = getTemporalSlice(start[i],end[i], ywhole,Xwhole)
            size = math.ceil(len(X)/16)
            between = np.random.choice(np.arange(len(X)), size=size, replace=False)
            X = np.array([X[i] for i in between])
            y = np.array([y[i] for i in between])
            if(len(X) == 0):
                continue
            Xtest,ytest = getTemporalSlice(start[i],end[i], ywhole5,Xwhole5)
            if(len(X) == 0 or len(Xtest) == 0):
                continue
            temporal_X, temporalY = getTemporalSlice(start[i],end[i], all_temporal_X,all_temporal_X)

            res, pred, std, opt_kernel = fitAndPlotGP(X,y,Xtest,ytest,kernel,temporal_X, graph=False)


            sumSqRes += res
            preds = np.concatenate([preds,pred])
            stds = np.concatenate([stds,std])
            Xtests = np.concatenate([Xtests,Xtest.flat])
            ytests = np.concatenate([ytests,ytest])
            times = np.concatenate([times,temporal_X.flat])
            Xs = np.concatenate([Xs,X.flat])
            ys = np.concatenate([ys,y]) 
            hyperparams = np.concatenate([hyperparams, np.array(opt_kernel.theta)[np.newaxis,:]])
        print("Squared Residuals", sumSqRes)
        meanSubtracted = np.sum((ywhole5-np.mean(ywhole5))**2)
        testCoef = (1-sumSqRes/meanSubtracted)
        print("Coefficient of determination:", testCoef)

#         res_list.append([windows, sumSqRes])
#         coef_list.append([windows, testCoef])

        if plotting:
            size = math.ceil(len(Xwhole)/16)
            between = np.random.choice(np.arange(len(Xwhole)), size=size, replace=False)
            testx = np.array([Xwhole[i] for i in between])
            testy = np.array([ywhole[i] for i in between])
            newTimes, preds = avg_group3d(times, preds)
            
            newTimes, stds = avg_group1d(times, stds)
            times = newTimes
            plt.figure(figsize=(24, 18), facecolor='w', edgecolor='k')
            plt.plot(Xwhole5,ywhole5[:,0], colors[2], label='test data', linewidth=5)
            plt.plot(Xwhole5,ywhole5[:,1], colors[2], linewidth=5)
            plt.plot(Xwhole5,ywhole5[:,2], colors[2], linewidth=5)

            plt.plot(testx, testy[:,0], 'o',color=colors[0], ms=5, label='x traning')
            plt.plot(testx, testy[:,1], 'v',color=colors[5], ms=5, label='y traning')
            plt.plot(testx, testy[:,2], 'X',color=colors[6], ms=5, label='z traning')

    #         plt.fill_between(times.flat, pred-1.96*std, pred+1.96*std, color=colors[3], alpha=.2)
            plt.fill_between(times.flat, preds[:,0]-1.96*stds, preds[:,0]+1.96*stds, color=colors[1], alpha=.2)
            plt.fill_between(times.flat, preds[:,1]-1.96*stds, preds[:,1]+1.96*stds, color=colors[3], alpha=.2)
            plt.fill_between(times.flat, preds[:,2]-1.96*stds, preds[:,2]+1.96*stds, color=colors[4], alpha=.2)

            plt.plot(times, preds[:,0], colors[1], lw=2, label='x mean', linewidth=5)
            plt.plot(times, preds[:,1], colors[3], lw=2, label='y mean', linewidth=5)
            plt.plot(times, preds[:,2], colors[4], lw=2, label='z mean', linewidth=5)


            plt.axis([0, times.flatten().max(), min(ys.min(),preds.min())-.2, max(ys.max(),preds.max())+.2])
#             plt.title('marker 8 using local kernel with {} windows\n'.format(windows) +'Sum of Squared Residuals: ' + str(sumSqRes) + '\n' + str(kernel),fontsize=40)
            plt.title('Marker 8 Using Local Kernel with Window Size {}, Delta {}'.format(window_size, delta) ,fontsize=20)

            plt.xlabel('time',fontsize=25)
            plt.ylabel('coordinate location',fontsize=25)
            plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1,  scatterpoints=1, prop={'size': 15},  title_fontsize=15)
#             for time_splits in range(windows):
#                 plt.axvline(x=time_splits*skips)
#             plt.savefig('marker_{}_3d_{}windows_12percent_sampled.png'.format(marker,windows))
            plt.show()
    


            def plotParams(hyperparams, param_names, start, end):
                plt.figure(figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
                all_temporal_X = np.arange(0, last_time,.05)
                all_params = []
                for i in range(len(all_temporal_X)):
                    sums = np.zeros(hyperparams.shape[-1])
                    count = 0
                    for window_num in range(min(len(end), len(start))):
#                         print('testing {} with range {} {}'.format(all_temporal_X[i], start[window_num], end[window_num]))

                        if start[window_num] <= all_temporal_X[i] and  all_temporal_X[i] < end[window_num]:
#                             print('in window num {} with range {} {}'.format(window_num, start[window_num], end[window_num]))
                            # check through hyperparams in this window and add them
                            sums += hyperparams[window_num+1]
                            count += 1
                        else:
                            continue
                    all_params.append(sums/count)
                    

                            
#                 for item in range(len(hyperparams[0])):
#                     print(hyperparams[1:,item])
#                     plt.plot(hyperparams[1:,item], label=param_names[item], linewidth=5)
#                 plt.xticks(np.arange(windows,step=1))
                plt.plot(all_temporal_X, np.array(all_params)[:,0], label=param_names[0], linewidth=5)
                plt.plot(all_temporal_X, np.array(all_params)[:,1], label=param_names[1], linewidth=5)
                plt.plot(all_temporal_X, np.array(all_params)[:,2], label=param_names[2], linewidth=5)
                plt.plot(Xwhole5,ywhole5*10, label='scaled trajectories', linewidth=5, c=colors[3])
                
                print(hyperparams[0])
                plt.axhline(hyperparams[0][0],label='init_'+param_names[0], linewidth=5, alpha=.6,c=colors[0])
                plt.axhline(hyperparams[0][1],label='init_'+param_names[1], linewidth=5, alpha=.6,c=colors[1])
                plt.axhline(hyperparams[0][2],label='init_'+param_names[2], linewidth=5, alpha=.6,c=colors[2])

                plt.xlabel('time',fontsize=25)
                handles, labels = plt.gca().get_legend_handles_labels()
                by_label = OrderedDict(zip(labels, handles))
                plt.legend(by_label.values(), by_label.keys(),loc="lower left",  scatterpoints=1, prop={'size': 15},  title_fontsize=15)

#                 plt.legend()
                plt.ylabel('hyperparameter values',fontsize=25)
                plt.title("Marker 8 Average Hyperparameters Over Time; Window Size {}, Delta {}".format(window_size, delta),fontsize=20)
#                 plt.savefig('{}_{}_{}windows_12percent_sampled_hyperparams.png'.format(marker,coord,windows))

                plt.show()
    
            
            param_names = ['sigma_f', 'sigma_l', 'sigma_n']
            plotParams(hyperparams,param_names,start,end)


In [ ]:
plt.ion()
%matplotlib

residuals = np.array(res_list)
# plt.scatter(residuals[:,0], residuals[:,1])


gpres = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=20, normalize_y=True)
gpres.fit(residuals[:,0].reshape(-1,1),residuals[:,1])


toPredict = np.arange(0,30,1).reshape(-1,1)

pred, std = gpres.predict(toPredict, return_std=True)


plt.figure(figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')

plt.plot(residuals[:,0], residuals[:,1], 'o',color=colors[0], ms=5, label='training points')

plt.fill_between(toPredict.flat, pred-1.96*std, pred+1.96*std, color=colors[3], alpha=.2)
plt.plot(residuals[:,0], pred, colors[3], lw=2, label='mean prediction', linewidth=5)
plt.axis([0, toPredict.flatten().max(), min(residuals[:,1].min(),pred.min())-.2, max(residuals[:,1].max(),pred.max())+.2])
plt.title('hi')

plt.xlabel('window',fontsize=25)
plt.ylabel('square residuals',fontsize=25)
plt.legend(loc="best",  scatterpoints=1, prop={'size': 15},  title_fontsize=15)






In [ ]:
coeffs = np.array(coef_list)
plt.scatter(coeffs[:,0], coeffs[:,1])

In [ ]:


all_preds = []
last_time = sorted(Xwhole.flat)[-1]

temporal_X = np.arange(0, last_time,.05).reshape(-1,1)

for marker in range(50):
    # Xwhole1, ywhole1 = get1Ddata(train_data1, train_times1, marker, coord)
    Xwhole2, ywhole2 = get3Ddata(train_data2, train_times2, marker)
    Xwhole4, ywhole4 = get3Ddata(train_data3, train_times3, marker)
    Xwhole5, ywhole5 = get3Ddata(train_data4, train_times4, marker)
    Xwhole3, ywhole3 = get3Ddata(test_data1, test_times1, marker)
    ywhole2 = ywhole2.reshape(-1,3)
    ywhole3 = ywhole3.reshape(-1,3)
    ywhole4 = ywhole4.reshape(-1,3)
    ywhole5 = ywhole5.reshape(-1,3)

    Xwhole = np.concatenate([Xwhole2,Xwhole3,Xwhole4,Xwhole5])
    ywhole = np.concatenate([ywhole2,ywhole3,ywhole4,ywhole5])
    print(marker, ywhole.shape, Xwhole.shape)

    if(len(Xwhole) == 0):
        all_preds.append([])
        continue
    size = math.ceil(len(Xwhole)/8)
    if len(Xwhole) <= 8:
        size = len(Xwhole)
    # size = 500
    between = np.random.choice(np.arange(len(Xwhole)), size=size, replace=False)

    X = np.array([Xwhole[i] for i in between])
    y = np.array([ywhole[i] for i in between])
    print(y.shape)
    kernel = ConstantKernel(constant_value=.25, constant_value_bounds=(0,10)) + RBF() + WhiteKernel(noise_level=3.0, noise_level_bounds=(0,10))

    sq_res, pred, std, opt_kernel = fitAndPlotGP(X,y,Xwhole,ywhole,kernel,temporal_X,graph=False)
    
    all_preds.append(pred)
plt.show()

In [ ]:
def normalize(X):
    a = X - np.nanmin(X)
    d = np.nanmax(X) - np.nanmin(X)
    return a/d


In [ ]:
# newPreds = np.array(len(all_preds[0]), 50, 3)
temp = all_preds.copy()
for i in range(len(all_preds)):
    if len(all_preds[i]) == 0:
        temp[i] = np.full(shape=(len(all_preds[0]),4),fill_value=-1)
    else:
        another = np.full(shape=(350,4),fill_value=1.0)
        another[:,:3] = all_preds[i]
        temp[i] = another
#         temp[i] = np.concatenate([all_preds[i], np.full(shape=(0,3),fill_value=-1)])
print(np.array(temp).shape)              

In [ ]:
frames = np.array(temp)
print(frames.shape).

In [ ]:
bone_list = [[39,40],[39,42],[38,39],[37,38],[36,37], \
             [46,49],[46,47],[45,46],[44,45],[43,44], \
             [36,43],[30,43],[30,36],[30,31],[31,32],[32,33], \
             [33,34],[33,35],[34,44],[35,37],[32,6],[32,18], \
             [18,19],[6,7], \
             [31,7],[7,8],[8,9],[9,15],[14,15],[15,10],[15,11],[15,12],[15,13],[15,16],[16,17], \
             [31,19],[19,20],[20,21],[21,27],[27,26],[26,22],[27,23],[27,24],[27,25],[27,28],[28,29], \
             [31,1],[1,0],[1,2],[1,3],[3,4],[4,5], [32,1]]
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.set_xlim3d([0,1])

# ax.set_xlim3d([-1,1])
ax.set_xlabel('X')
ax.set_ylim3d([0,1])

# ax.set_ylim3d([2,3])
ax.set_ylabel('Y')

ax.set_zlim3d([0,1])

# ax.set_zlim3d([-1,1])
ax.set_zlabel('Z')

# data = np.array((normalize(frames[0,:,2]), \
#            normalize(frames[0,:,0]), \
#            normalize(frames[0,:,1])))


line_data = []
for it in range(0,1030,2):
#     ax = p3.Axes3D(fig)
    ax.set_xlim3d([0,1])
    ax.set_xlabel('X')
    ax.set_ylim3d([0,1])
    ax.set_ylabel('Y')
    ax.set_zlim3d([0,1])
    ax.set_zlabel('Z')
    data = np.array((normalize(frames[it,:,2]), \
           normalize(frames[it,:,0]), \
           normalize(frames[it,:,1])))
    for i, bone in enumerate(bone_list):
        if frames[it,bone[0],3] <= 0 or frames[it,bone[1],3] <= -1:
            continue
        x,y,z = [np.array( [ data[j,bone[0]], data[j, bone[1]] ] ) for j in range(3)]
        ax.plot(x,y,z)
    ax.scatter(normalize(frames[it,:,2]), \
       normalize(frames[it,:,0]), \
       normalize(frames[it,:,1]) )
#     ax.scatter(normalize(frames[it,:,2])[20], \
#        normalize(frames[it,:,0])[20], \
#        normalize(frames[it,:,1])[20] )
    plt.savefig('anim/test{}.png'.format(it))
    ax.clear()

print(data[0].shape)